In [11]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../../data/'
group_dir_ = os.path.join(dir_, 'groups/high')
folder = 'sp_matrix_tag_pop'
file_name = 'normalized_to_rating_filter_track_5_user_50.pkl'

In [12]:
all_user = pd.read_pickle(os.path.join(dir_, folder, file_name[:-3] + 'pkl'))
predictions = pd.read_pickle(os.path.join(dir_, folder, 'topN_pred_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, folder, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,count,tags,rating,pop
7,0,2,4,0,3.00,24
24,0,14,2,1,1.50,6
25,0,15,12,0,4.75,53
33,0,20,3,12,2.50,12
34,0,21,1,5,0.25,6


In [13]:
predictions[:5]

,uid,tid,rating
0,0,2,0.070515
1,0,14,0.022937
2,0,15,0.147609
3,0,20,0.038996
4,0,21,0.023123


In [14]:
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,count,tags,rating,pop
7,0,2,4,0,3.00,24
24,0,14,2,1,1.50,6
25,0,15,12,0,4.75,53
33,0,20,3,12,2.50,12
34,0,21,1,5,0.25,6


In [15]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10', '20']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [16]:
num_user = len(predictions['uid'].unique())
num_user

218

In [17]:
pop_dict = {}
for i in all_user['tid'].unique():
    pop_dict[i] = len(all_user[all_user['tid']==i])/num_user

In [18]:
pop_4 = []
pop_20 = []
pop_50 = []

In [19]:
top_n_size = 50
top_n_lists_g = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([50, 20, 4]):
    avg_pops = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        pops =[]
        for j in (range(len(groups))):
            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    tid = top_n_list[l]
                    pops.append(pop_dict[tid])
        avg_pop = sum(pops)/len(pops)
        avg_pops.append(avg_pop)
        
    if top_n_size == 4:
        pop_4.append(avg_pops)
    elif top_n_size == 20:
        pop_20.append(avg_pops)
    elif top_n_size == 50:
        pop_50.append(avg_pops)

In [20]:
for i in pop_4:
    print(i)
for i in pop_20:
    print(i)
for i in pop_50:
    print(i)

[0.3863435392947953, 0.3705592426312714, 0.36001265422334827, 0.3519078398665565, 0.3470183486238534, 0.33648700305810364, 0.33428899082568736, 0.32568807339449485, 0.31498470948012197, 0.3497706422018344]
[0.3492787664419121, 0.3300946710911561, 0.3259965200885791, 0.3231755629691407, 0.31755351681957206, 0.30963302752293675, 0.3075458715596333, 0.3049311926605507, 0.29820336391437235, 0.30779816513761377]
[0.31910246490549654, 0.30328128049970615, 0.30241379310344674, 0.30016680567139187, 0.2956513761467896, 0.29136850152904686, 0.28897247706421864, 0.2865596330275198, 0.2781804281345547, 0.29137614678899]
